## Generación de los Datos

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

torch.manual_seed(42)

X = torch.randint(1, 10, (1, 1, 6, 6)).float()
print("="*60)
print("X shape:", X.shape)
print("Una Imagen, de 1 canal de tamaño 6x6")
print("="*60)
X

X shape: torch.Size([1, 1, 6, 6])
Una Imagen, de 1 canal de tamaño 6x6


tensor([[[[7., 6., 8., 5., 1., 3.],
          [8., 6., 5., 3., 5., 5.],
          [9., 1., 1., 5., 3., 5.],
          [4., 5., 5., 9., 2., 6.],
          [9., 5., 3., 1., 2., 2.],
          [4., 4., 8., 8., 9., 8.]]]])

In [2]:
print("="*60)
print("Hiperparámetros de la Convolución")
C_out = 2
N, C_in, H, W = X.shape
kH, kW = (3,3)

print("="*60)
print(f"Número de Feature Maps (C_out): {C_out}")
print(f"Tamaño del Kernel de la Convolución: {kH,kW}")


Hiperparámetros de la Convolución
Número de Feature Maps (C_out): 2
Tamaño del Kernel de la Convolución: (3, 3)


In [3]:
print("="*60)
print("Parámetros de la convolución: ")
print("Se va a aplicar 2 filtros de tamaño 3x3 y un bias para cada filtro")
print("="*60)
given_w = torch.randint(-1,2, (C_out, C_in, kH,kW)).float()
given_bias = torch.tensor([1.,1.]) # (2,)
print("W_conv: ", end="")
print(given_w)
print("="*60)
print("bias_conv: ", end="")
print(given_bias)
print("="*60)


Parámetros de la convolución: 
Se va a aplicar 2 filtros de tamaño 3x3 y un bias para cada filtro
W_conv: tensor([[[[ 1.,  1.,  0.],
          [-1.,  1., -1.],
          [ 0., -1., -1.]]],


        [[[ 1.,  1.,  0.],
          [-1.,  1., -1.],
          [-1.,  0., -1.]]]])
bias_conv: tensor([1., 1.])


## Pytorch `nn.Module`

In [4]:
print("="*60)
print("Definición del Modelo en Pytorch utilizando nn.Module")
print("="*60)
class Conv(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, bias=True)
        self.conv.weight.data = given_w
        self.conv.bias.data = given_bias
        self.max_pool = nn.MaxPool2d(kernel_size=2, return_indices=True)
        self.fc = nn.Linear(8, 1)
        nn.init.ones_(self.fc.weight)
        nn.init.ones_(self.fc.bias)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv(x)
        print(x)
        x, self.indices = self.max_pool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

model = Conv()
criterion = nn.BCEWithLogitsLoss()
y = torch.zeros([N,1])

# Forward con PyTorch (referencia)
logits = model(X)
loss = criterion(logits, y)

print("="*60)
print("y: ", end="")
print(y)
print("="*60)
print(f"Loss obtenido: {loss}")
print("="*60)
print(f"Logits: {logits}")
print("="*60)

Definición del Modelo en Pytorch utilizando nn.Module
tensor([[[[  5.,   5.,  -1.,  -4.],
          [ -4.,  -7.,  -1.,  -6.],
          [ -1., -10.,   6.,  -8.],
          [ -9.,  -8.,  -6.,  -6.]],

         [[ -3.,   5.,   3.,  -6.],
          [ -3.,  -7.,   3., -13.],
          [ -5., -12.,   4.,  -7.],
          [ -9.,  -4.,  -6.,  -5.]]]], grad_fn=<ConvolutionBackward0>)
y: tensor([[0.]])
Loss obtenido: 18.0
Logits: tensor([[18.]], grad_fn=<AddmmBackward0>)


## Forward Pass Manual

In [5]:
def calculate_out(X, k_size=(3,3), stride=1, dilation=1, padding=0):
  kH, kW = k_size
  N, in_channels, H_in, W_in = X.shape
  out_H = np.floor((H_in +2*padding-dilation*(kH-1)-1)/stride + 1)
  out_W = np.floor((W_in +2*padding-dilation*(kW-1)-1)/stride + 1)
  return int(out_H), int(out_W)

H_out, W_out = calculate_out(X, k_size = (kH,kW))
print("="*60)
print(f"Tamaño resultante post-convolución: {H_out, W_out}")
print("="*60)

Tamaño resultante post-convolución: (4, 4)


In [6]:
print("="*60)
print("Implementación básica de la Convolución: Muy Ineficiente")
print("="*60)
O = torch.zeros((N, C_out, H_out, W_out))
for n in range(N):
    for co in range(C_out):
        for i in range(H_out):
            for j in range(W_out):
                patch = X[n, :, i:i+kH, j:j+kW]       # submatriz de tamaño kH x kW
                O[n, co, i, j] = (patch * given_w[co]).sum() + given_bias[co]


print("O:", end="")
print(O)
print("="*60)

Implementación básica de la Convolución: Muy Ineficiente
O:tensor([[[[  5.,   5.,  -1.,  -4.],
          [ -4.,  -7.,  -1.,  -6.],
          [ -1., -10.,   6.,  -8.],
          [ -9.,  -8.,  -6.,  -6.]],

         [[ -3.,   5.,   3.,  -6.],
          [ -3.,  -7.,   3., -13.],
          [ -5., -12.,   4.,  -7.],
          [ -9.,  -4.,  -6.,  -5.]]]])


In [7]:
# im2col: Convierte ventanas deslizantes en columnas
print("="*60)
print("Implementación Algoritmo im2col: ")
print("="*60)
X_col = F.unfold(X, kernel_size=(kH, kW))  # (1, 9, 16) (N,kH*kW,n_patches)
print(f"Cada columna es una ventana 3x3 aplanada")
print(f"Tenemos 16 patches (4x4 posiciones de salida) para una imagen")
print("X_col: ", end="")
print(X_col)
print("="*60)
print(f"X_col shape: {X_col.shape}")
print("="*60)


Implementación Algoritmo im2col: 
Cada columna es una ventana 3x3 aplanada
Tenemos 16 patches (4x4 posiciones de salida) para una imagen
X_col: tensor([[[7., 6., 8., 5., 8., 6., 5., 3., 9., 1., 1., 5., 4., 5., 5., 9.],
         [6., 8., 5., 1., 6., 5., 3., 5., 1., 1., 5., 3., 5., 5., 9., 2.],
         [8., 5., 1., 3., 5., 3., 5., 5., 1., 5., 3., 5., 5., 9., 2., 6.],
         [8., 6., 5., 3., 9., 1., 1., 5., 4., 5., 5., 9., 9., 5., 3., 1.],
         [6., 5., 3., 5., 1., 1., 5., 3., 5., 5., 9., 2., 5., 3., 1., 2.],
         [5., 3., 5., 5., 1., 5., 3., 5., 5., 9., 2., 6., 3., 1., 2., 2.],
         [9., 1., 1., 5., 4., 5., 5., 9., 9., 5., 3., 1., 4., 4., 8., 8.],
         [1., 1., 5., 3., 5., 5., 9., 2., 5., 3., 1., 2., 4., 8., 8., 9.],
         [1., 5., 3., 5., 5., 9., 2., 6., 3., 1., 2., 2., 8., 8., 9., 8.]]])
X_col shape: torch.Size([1, 9, 16])


In [8]:
print("="*60)
print("Implementación de la Convolución utilizando im2col:")
print("="*60)
# Reshape de pesos

W_row = given_w.reshape(C_out, -1)  # (2, 9)
print(f"Cada fila es un filtro 3x3 aplanado")
print(f"W_row shape: {W_row.shape}")
print("W_row: ", end="")
print(W_row)
print("="*60)


# Multiplicación matricial y corregimos dimensiones...
H_col = W_row @ X_col + given_bias.reshape(-1, 1)  # (1,2,16) # (N, C_out, n_patches)
_, _, n_patches = H_col.shape
H = H_col.reshape(N, C_out, H_out, W_out)  # (1, 2, 4, 4)
print(f"H shape: {H.shape}")
print("="*60)
print("Corresponde a un imagen de 2 Feature Maps de Salida.")
print("Cada Feature Map es de 4x4")
print("="*60)
print("H: ", end="")
print(H)
print("="*60)


Implementación de la Convolución utilizando im2col:
Cada fila es un filtro 3x3 aplanado
W_row shape: torch.Size([2, 9])
W_row: tensor([[ 1.,  1.,  0., -1.,  1., -1.,  0., -1., -1.],
        [ 1.,  1.,  0., -1.,  1., -1., -1.,  0., -1.]])
H shape: torch.Size([1, 2, 4, 4])
Corresponde a un imagen de 2 Feature Maps de Salida.
Cada Feature Map es de 4x4
H: tensor([[[[  5.,   5.,  -1.,  -4.],
          [ -4.,  -7.,  -1.,  -6.],
          [ -1., -10.,   6.,  -8.],
          [ -9.,  -8.,  -6.,  -6.]],

         [[ -3.,   5.,   3.,  -6.],
          [ -3.,  -7.,   3., -13.],
          [ -5., -12.,   4.,  -7.],
          [ -9.,  -4.,  -6.,  -5.]]]])


In [9]:
H_pool, W_pool = calculate_out(H, k_size=(2,2),stride=2)
print("="*60)
print(f"Tamaño resultante post-pooling: {H_pool, W_pool}")
print("="*60)

Tamaño resultante post-pooling: (2, 2)


In [10]:
print("="*60)
print("Forward Pass Pooling usando im2col: ")
print("="*60)
pool_size = 2

h_col = F.unfold(H, kernel_size=pool_size, stride=pool_size)  # (1, 8, 4)
print(f"Shape h_col: {h_col.shape}")
print("="*60)
print("h_col: ", end="")
print(h_col)
print("="*60)
h_col_reshaped = h_col.view(N, C_out, pool_size*pool_size, -1)  # (1, 2, 4, 4)
print(f"Shape h_col_reshaped: {h_col_reshaped.shape}")
print("="*60)
print(h_col_reshaped)
print("="*60)
M_flat, pool_indices = h_col_reshaped.max(dim=2)
print("="*60)
print("M_flat: ", end="")
print(M_flat)
print("="*60)
M = M_flat.reshape(N, C_out, H_pool, W_pool)
print("Reconstrucción a Feature Map")
print("M: ", end="")
print(M)
print("="*60)

Forward Pass Pooling usando im2col: 
Shape h_col: torch.Size([1, 8, 4])
h_col: tensor([[[  5.,  -1.,  -1.,   6.],
         [  5.,  -4., -10.,  -8.],
         [ -4.,  -1.,  -9.,  -6.],
         [ -7.,  -6.,  -8.,  -6.],
         [ -3.,   3.,  -5.,   4.],
         [  5.,  -6., -12.,  -7.],
         [ -3.,   3.,  -9.,  -6.],
         [ -7., -13.,  -4.,  -5.]]])
Shape h_col_reshaped: torch.Size([1, 2, 4, 4])
tensor([[[[  5.,  -1.,  -1.,   6.],
          [  5.,  -4., -10.,  -8.],
          [ -4.,  -1.,  -9.,  -6.],
          [ -7.,  -6.,  -8.,  -6.]],

         [[ -3.,   3.,  -5.,   4.],
          [  5.,  -6., -12.,  -7.],
          [ -3.,   3.,  -9.,  -6.],
          [ -7., -13.,  -4.,  -5.]]]])
M_flat: tensor([[[ 5., -1., -1.,  6.],
         [ 5.,  3., -4.,  4.]]])
Reconstrucción a Feature Map
M: tensor([[[[ 5., -1.],
          [-1.,  6.]],

         [[ 5.,  3.],
          [-4.,  4.]]]])


In [11]:
print("="*60)
print("Flatten")
print("="*60)
f = M.view(N, -1)  # (1, 8)
print(f"Shape post-Flatten: {f.shape}")


print("="*60)
print("Feed Forward")
print("="*60)
print("Parámetros: ")
print("W_fc: ", end="")
W_fc = torch.ones(8, 1)
print(W_fc)
b_fc = torch.ones(1)
print("="*60)
print("b_fc: ", end="")
print(b_fc)
Z = f @ W_fc + b_fc
print("="*60)
print("Logits: ", end="")
print(Z)
print("="*60)



Flatten
Shape post-Flatten: torch.Size([1, 8])
Feed Forward
Parámetros: 
W_fc: tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])
b_fc: tensor([1.])
Logits: tensor([[18.]])


## Backward Pass Manual

In [12]:
print("="*60)
print("Debido a que partimos derivando desde atrás, \nlas derivadas son muy sencillas y se realizan de forma idéntica a lo visto en clases")
print("Sólo nos interesa calcular las derivadas respecto a W_conv que son parámetros que no conocemos...")
print("="*60)
print("Derivada del BCEwithLogitLoss: ")
dZ = torch.sigmoid(Z) - y  # (1, 1)
print("dZ: ", dZ)
print("="*60)
print("Derivada hasta f")
df = dZ @ W_fc.T           # (1, 8)
print("df: ", df)
print("="*60)
print("El gradiente hasta M es el inverso del Flatten, es decir recobramos la forma original")

dM = df.reshape(N, C_out, pool_size, pool_size)
print("dM: ", dM)
print(f"Shape de dM: {dM.shape}")
print("Una imagen de 2 canales de tamaño 2x2")
print("="*60)

Debido a que partimos derivando desde atrás, 
las derivadas son muy sencillas y se realizan de forma idéntica a lo visto en clases
Sólo nos interesa calcular las derivadas respecto a W_conv que son parámetros que no conocemos...
Derivada del BCEwithLogitLoss: 
dZ:  tensor([[1.]])
Derivada hasta f
df:  tensor([[1., 1., 1., 1., 1., 1., 1., 1.]])
El gradiente hasta M es el inverso del Flatten, es decir recobramos la forma original
dM:  tensor([[[[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]]]])
Shape de dM: torch.Size([1, 2, 2, 2])
Una imagen de 2 canales de tamaño 2x2


In [13]:
print("="*60)
print("Gradiente del Pooling hasta h_col")
dh_col = torch.zeros_like(h_col_reshaped)
dh_col.scatter_(2, pool_indices.unsqueeze(2), dM.reshape(N, C_out, 1, -1))
print("="*60)
print("h_col:", h_col_reshaped)
print("="*60)
print("Se puede notar que el gradiente del Max Pooling coloca un 1 en la fila de cada columna en la que se encontró el máximo y cero en otro caso")
print("Para poder restaurar esta información fue necesario guardar los índices del Máximo en el Forward Pass..")
print("dh_col: ", dh_col)
print("="*60)

Gradiente del Pooling hasta h_col
h_col: tensor([[[[  5.,  -1.,  -1.,   6.],
          [  5.,  -4., -10.,  -8.],
          [ -4.,  -1.,  -9.,  -6.],
          [ -7.,  -6.,  -8.,  -6.]],

         [[ -3.,   3.,  -5.,   4.],
          [  5.,  -6., -12.,  -7.],
          [ -3.,   3.,  -9.,  -6.],
          [ -7., -13.,  -4.,  -5.]]]])
Se puede notar que el gradiente del Max Pooling coloca un 1 en la fila de cada columna en la que se encontró el máximo y cero en otro caso
Para poder restaurar esta información fue necesario guardar los índices del Máximo en el Forward Pass..
dh_col:  tensor([[[[1., 1., 1., 1.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 1., 0., 1.],
          [1., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 1., 0.]]]])


In [14]:
print("="*60)
print("Gradiente hasta H")
print("Debido a que pasamos de h_col a H, es necesario calcular el Gradiente el im2col")
print("El gradiente es el Algoritmo inverso: col2im")
print("="*60)
print("Primero debemos juntar los canales, que es lo que es pera de resultado el im2col")
dh_col_flat = dh_col.reshape(N, C_out * 4, -1)
print(dh_col_flat)
print("="*60)
print("Aplicamos col2im: ")
print("Acá debemos reconstruir utilizando el Kernel y el Stride utilizado (Los del Pooling)")
print("Luego debemos ingresar el tamaño resultante, que sería el de la entrada al Pooling (Salida de la Convolución)")
dH = F.fold(dh_col_flat, output_size=(H_out, W_out), kernel_size=2, stride=2)
print("dH", dH)
print("="*60)
print(f"Shape dH: {dH.shape}")
print("="*60)

dH_col = dH.view(N, C_out, -1)  # (N, C_out, num_patches)

Gradiente hasta H
Debido a que pasamos de h_col a H, es necesario calcular el Gradiente el im2col
El gradiente es el Algoritmo inverso: col2im
Primero debemos juntar los canales, que es lo que es pera de resultado el im2col
tensor([[[1., 1., 1., 1.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 1., 0., 1.],
         [1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 1., 0.]]])
Aplicamos col2im: 
Acá debemos reconstruir utilizando el Kernel y el Stride utilizado (Los del Pooling)
Luego debemos ingresar el tamaño resultante, que sería el de la entrada al Pooling (Salida de la Convolución)
dH tensor([[[[1., 0., 1., 0.],
          [0., 0., 0., 0.],
          [1., 0., 1., 0.],
          [0., 0., 0., 0.]],

         [[0., 1., 1., 0.],
          [0., 0., 0., 0.],
          [0., 0., 1., 0.],
          [0., 1., 0., 0.]]]])
Shape dH: torch.Size([1, 2, 4, 4])


In [15]:
print("="*60)
print("El gradiente de W_conv es similar al de una Linear Layer pero con un truquito de shapes para poder hacer multiplicaciones válidas...")
print("="*60)
dH_reshaped = dH.permute(1,0,2,3).reshape(C_out, -1) # (C_out, N*n_patches)
X_reshaped = X_col.reshape(C_in*kH*kW,-1) # (C_in*kH*kW, N*n_patches)
print("="*60)
dW_conv_flat = dH_reshaped @ X_reshaped.T  # (C_out, C_in*kH*kW)
print("dW_conv_flat: ", dW_conv_flat)
print("="*60)
dW_conv = dW_conv_flat.reshape(C_out, C_in, kH, kW)
print("dW_conv: ", dW_conv)
print("="*60)


El gradiente de W_conv es similar al de una Linear Layer pero con un truquito de shapes para poder hacer multiplicaciones válidas...
dW_conv_flat:  tensor([[25., 17., 13., 22., 23., 17., 22., 12.,  9.],
        [20., 23., 18., 21., 20., 11.,  9., 15., 18.]])
dW_conv:  tensor([[[[25., 17., 13.],
          [22., 23., 17.],
          [22., 12.,  9.]]],


        [[[20., 23., 18.],
          [21., 20., 11.],
          [ 9., 15., 18.]]]])


In [16]:
print("="*60)
print("El Gradiente del Bias: ")
print("="*60)
dBias_conv = torch.ones(n_patches)@dH_col.transpose(-1,-2)
print("dbias_conv: ", dBias_conv)


El Gradiente del Bias: 
dbias_conv:  tensor([[4., 4.]])


## Gradientes calculados con Pytorch `nn.Module`

In [17]:
loss.backward()
print("="*60)
print("Gradientes Calculados utilizando nn.Module")
print("="*60)
print("dW_conv_pytorch: ",  model.conv.weight.grad)
print("="*60)
print("dbias_conv_pytorch: ",  model.conv.bias.grad)
print("="*60)

Gradientes Calculados utilizando nn.Module
dW_conv_pytorch:  tensor([[[[25., 17., 13.],
          [22., 23., 17.],
          [22., 12.,  9.]]],


        [[[20., 23., 18.],
          [21., 20., 11.],
          [ 9., 15., 18.]]]])
dbias_conv_pytorch:  tensor([4., 4.])
